### Tennis-Player (part of the Collaboration and Competition-Project)

With this Tennis-Player program, it is possible to choos a players level.
The Idea behind is, to show how the players behaviour change at different
training levels.
#### Training-Levels:
- player level = 1 -->  poor players 
    - player_1 with score of 0.1
    - player_2 with score of 0.1
    
- player level = 2 -->  mixed players  
    - player_1 with score of 0.1
    - player_2 with score of 0.5

- player level = 3 -->  mixed players  
    - player_1 with score of 0.5
    - player_2 with score of 0.5
 






####  1. Start the Environment

In [1]:
#!pip -q install ./python

In [2]:
from unityagents import UnityEnvironment
import numpy as np

In [3]:
env = UnityEnvironment(file_name="Tennis.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


#### 2. Get some Tennis-Player environment data

In [5]:
# reset the environment
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
env_info = env.reset(train_mode=True)[brain_name]
# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
env_info = env.reset(train_mode=True)[brain_name]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])


import numpy as np
import random
import copy
import gym

import torch
import matplotlib.pyplot as plt
%matplotlib inline
from collections import deque
from collections import namedtuple, deque
from maddpg_agent import Agent 


Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.69487906 -1.5
 -0.          0.          6.83172083  5.98822832 -0.          0.        ]


#### 3. Method for loading the level-weights: "set_player_performance": 



In [6]:
# player-level
# 1 = poor players
# 2 = mixed players
# 3 = good players

def set_player_performance (player_performance,agent_player_1, agent_player_2):

# player level = 1 -->  poor players
# player level = 2 -->  mixed players
# player level = 3 -->  good players#

    if player_performance == 1: 
        # Set 1 -->  bad_players: Player_1: score = 0.1
        #                         Player_2: score = 0.1
        agent_player_1.actor_local.load_state_dict(torch.load('score_01_player_1_checkpoint_actor.pth', map_location='cpu'))
        agent_player_2.actor_local.load_state_dict(torch.load('score_01_player_2_checkpoint_actor.pth', map_location='cpu'))
        print("--level 1--")                                           
    elif player_performance == 2:
        # Set 2 with mixed_players: Player_1: score = 0.1
        #                           Player_2: score = 0.5    
        agent_player_1.actor_local.load_state_dict(torch.load('score_01_player_1_checkpoint_actor.pth', map_location='cpu'))
        agent_player_2.actor_local.load_state_dict(torch.load('score_03_player_2_checkpoint_actor.pth', map_location='cpu'))
        print("--level 2--")  
    elif player_performance == 3:
        # Set 3 with good players: Player_1: score = 0.5
        #                          Player_2: score = 0.5
        agent_player_1.actor_local.load_state_dict(torch.load('score_05_player_1_checkpoint_actor.pth', map_location='cpu'))
        agent_player_2.actor_local.load_state_dict(torch.load('score_05_player_2_checkpoint_actor.pth', map_location='cpu'))
        print("--level 3--")  
    else: 
        print("Player Performance-Set is not valid.")   



#### 4. Methode of the Tennis-Player:  "tennis_player"

In [7]:
def tennis_player (max_time_step=1000):
   
    #env = UnityEnvironment(file_name="Tennis.exe")
    
    brain_name = env.brain_names[0]
    brain = env.brains[brain_name]
    env_info = env.reset(train_mode=False)[brain_name]
    
    # number of agents 
    num_agents = len(env_info.agents)
    # print('Number of agents:', num_agents)

    # size of each action
    action_size = brain.vector_action_space_size
    # print('Size of each action:', action_size)

    # examine the state space 
    states = env_info.vector_observations
    state_size = states.shape[1]
    
    #create agent and run against environment
    agent_player_1 = Agent(state_size=state_size, action_size=action_size, num_agents=num_agents, random_seed=24)
    agent_player_2 = Agent(state_size=state_size, action_size=action_size, num_agents=num_agents, random_seed=24)
    
    agent_player_1.reset()
    agent_player_2.reset()
    
    print("Please input the player-level:")
    print("For poor players  --> input: 1")
    print("For mixed players --> input: 2")
    print("For good players  --> input: 3")

    player_performance = int(input())
    
    set_player_performance (player_performance, agent_player_1, agent_player_2)
    
    #agent_player_2.critic_local.load_state_dict(torch.load('score_05_player_1_checkpoint_critic.pth', map_location='cpu')))
    #agent_player_2.critic_local.load_state_dict(torch.load('score_05_player_2_checkpoint_critic.pth', map_location='cpu')))
   
    add_noise = False
    count_steps = 0

    for t in range(max_time_step):
        # actions = agent.act(states)
        actions = np.random.randn(num_agents, action_size)
        actions[0] = agent_player_1.act(states[0], add_noise) 
        actions[1] = agent_player_2.act(states[1], add_noise)
            
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent) 
        states = next_states 
        count_steps  += 1     

        if (count_steps > max_time_step):
            env.close()
            print("Tennis player has stopped")
            break


#### 5. Run the Tennis-Player by entering the the players level (1,2 or 3) in a input-field
The Tennis-Player runs for 500 Timesteps.
The Timestep can be changed by changing the input-parameter of tennis_player-Methode 

In [ ]:
tennis_player(1000)

Please input the player-level:
For poor players  --> input: 1
For mixed players --> input: 2
For good players  --> input: 3
2
--level 2--


D:\Miniconda\envs\drlnd\lib\site-packages\torch\nn\functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
